# Custom Map File Creation notebook

This notebook will develop code to create the files that are necessary in the Beat Saber custom songs directory:

- .dat files (info.dat, 'level'.dat)
- cover.jpg file
- song.egg file